In [24]:
import pandas as pd
import numpy as np

from prophet import Prophet
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


In [25]:
accomodation_foreigners = pd.read_csv("accommodation/accomodation_foreigner.csv", skiprows=1)
accomodation_total = pd.read_csv("accommodation/accomodation_total.csv", skiprows=1)

#prefectures = pd.read_csv("Prefecture_Japanese_Area.tsv", sep="\t", thousands=",")



In [26]:
prefectures = {}

is_header = True
for line in open("Prefecture_Japanese_Area.tsv", 'r'):
    if is_header == True:
        is_header = False
    else:
        fields = line.strip().split("\t")
        prefectures[fields[1]] = fields[0]

In [27]:
accomodation_foreigners.rename(columns = prefectures, inplace=True)
accomodation_foreigners.rename(columns = {"時点": "time"}, inplace=True)

accomodation_total.rename(columns = prefectures, inplace=True)
accomodation_total.rename(columns = {"時点": "time"}, inplace=True)


In [28]:
accomodation_foreigners["time"] = accomodation_foreigners["time"].str.replace('年','-').str.replace('月','')
accomodation_foreigners = accomodation_foreigners[~accomodation_foreigners["time"].str.endswith("-")]
accomodation_foreigners["time"] = pd.to_datetime(accomodation_foreigners["time"].str.replace('年','-').str.replace('月',''), format='%Y-%m')

accomodation_total["time"] = accomodation_total["time"].str.replace('年','-').str.replace('月','')
accomodation_total = accomodation_total[~accomodation_total["time"].str.endswith("-")]
accomodation_total["time"] = pd.to_datetime(accomodation_total["time"].str.replace('年','-').str.replace('月',''), format='%Y-%m')

In [29]:
foreigners = pd.melt(accomodation_foreigners, id_vars=['time'], value_vars=[x for x in accomodation_foreigners.columns if x != "time"], var_name='location', value_name='foreigners')

total = pd.melt(accomodation_total, id_vars=['time'], value_vars=[x for x in accomodation_total.columns if x != "time"], var_name='location', value_name='total')

In [30]:
foreigners.head()

,time,location,foreigners
0,2007-01-01,Japan,1451290
1,2007-02-01,Japan,1749830
2,2007-03-01,Japan,1783830
3,2007-04-01,Japan,2106150
4,2007-05-01,Japan,1834840


In [31]:
total.head()

,time,location,total
0,2007-01-01,Japan,22136270
1,2007-02-01,Japan,23346870
2,2007-03-01,Japan,26997200
3,2007-04-01,Japan,23729790
4,2007-05-01,Japan,25636730


In [32]:
foreigners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8160 entries, 0 to 8159
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   time        8160 non-null   datetime64[ns]
 1   location    8160 non-null   object        
 2   foreigners  8160 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 191.4+ KB


In [33]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8160 entries, 0 to 8159
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   time      8160 non-null   datetime64[ns]
 1   location  8160 non-null   object        
 2   total     8160 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 191.4+ KB


In [34]:
table = pd.merge(total, foreigners, left_on=["time", "location"], right_on=["time", "location"], how='left')

In [35]:
table.head()

,time,location,total,foreigners
0,2007-01-01,Japan,22136270,1451290
1,2007-02-01,Japan,23346870,1749830
2,2007-03-01,Japan,26997200,1783830
3,2007-04-01,Japan,23729790,2106150
4,2007-05-01,Japan,25636730,1834840


In [36]:
table.to_csv("accomodation_total_and_foreigner.tsv", sep="\t", index=False)

In [37]:
prophet_japan = table[table["location"] == "Japan"][["time", "total"]]
prophet_japan.columns = ["ds", "y"]
prophet_japan["ds"] = pd.to_datetime(prophet_japan["ds"])

In [38]:
prophet_japan_2020 = prophet_japan[(prophet_japan["ds"] >= "2020-01-01")]
prophet_japan_before_2020 = prophet_japan[(prophet_japan["ds"] < "2020-01-01")]
m_japan = Prophet()
m_japan.fit(prophet_japan_before_2020)
future = pd.DataFrame(prophet_japan["ds"], columns=["ds"])
prediction = m_japan.predict(future)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [57]:
def plot_prophet(original, prediction, title, y_title):
    trace1 = go.Scatter(
    x=original["ds"],
    y=original["y"],
    mode='markers',
    name = "Actual",
    marker_color='rgba(152, 0, 0, .8)',
    marker={'size': 15}
    )

    trace2 = go.Scatter(
        x=prediction["ds"],
        y=prediction["yhat"],
        name = "Prophet model",
        marker_color='rgb(129, 119, 185, 1)'
    )

    trace3 = go.Scatter(
        x=prediction["ds"],
        y=prediction["yhat_upper"],
        mode='lines',
            marker=dict(color="#444"),
            line=dict(width=0),
            showlegend=False
    )

    trace4 = go.Scatter(
        x=prediction["ds"],
        y=prediction["yhat_lower"],
        marker=dict(color="#444"),
            line=dict(width=0),
            mode='lines',
            fillcolor='rgba(78, 119, 185, 0.4)',
            fill='tonexty',
            showlegend=False
    )



    fig = make_subplots()
    
    fig.add_trace(trace2)
    fig.add_trace(trace3)
    fig.add_trace(trace4)
    fig.add_trace(trace1)
    #fig.write_image("fig1.png")
    fig.update_layout(height=800, width=1600, title_text=title,  xaxis_title="Time", yaxis_title=y_title, font=dict(size=22))
    return fig

In [58]:
fig_1 = plot_prophet(prophet_japan, prediction, "Accommodation counts in Japan between 2007 and early 2021", "Accomodation")
fig_1.show()

In [41]:
def potential_loss(prediction, actual, time_start, time_stop):
    temp_df = prediction[(prediction["ds"] >= time_start) & (prediction["ds"] <= time_stop)][["ds", "yhat"]]
    temp_df = temp_df.merge(prophet_japan[(prophet_japan["ds"] >= time_start) & (prophet_japan["ds"] <= time_stop)][["ds", "y"]], left_on="ds", right_on="ds")
    temp_df["diff"] = temp_df["yhat"] - temp_df["y"]

    return temp_df["diff"].sum()

In [42]:
potential_loss(prediction, prophet_japan, "2020-01-01", "2020-12-31")

308225212.84362864

In [43]:
def yoy(df):
    df['previous_year'] = df.groupby([df['ds'].dt.month, df['ds'].dt.day])['y'].shift()
    df["yoy"] = 100 * (df["y"] - df["previous_year"]) / df["previous_year"]
    return df

In [44]:
df = yoy(prophet_japan)

In [45]:
df[(df["ds"] > "2019")]

,ds,y,previous_year,yoy
145,2019-02-01,43539370,38899650.0,11.927408
146,2019-03-01,51147600,45644860.0,12.055552
147,2019-04-01,50718730,42509270.0,19.312164
148,2019-05-01,51402690,44445160.0,15.654190
149,2019-06-01,45810390,41406270.0,10.636360
150,2019-07-01,51780530,47142860.0,9.837481
151,2019-08-01,63234040,59715640.0,5.891924
152,2019-09-01,48761240,44439890.0,9.724034
153,2019-10-01,50052850,46582920.0,7.448932
154,2019-11-01,49659370,45282650.0,9.665335


In [46]:
table.head()

,time,location,total,foreigners
0,2007-01-01,Japan,22136270,1451290
1,2007-02-01,Japan,23346870,1749830
2,2007-03-01,Japan,26997200,1783830
3,2007-04-01,Japan,23729790,2106150
4,2007-05-01,Japan,25636730,1834840


In [47]:

df = table[table["location"].isin(["Tokyo",  "Osaka", "Hokkaidou",  "Okinawa", "Kyoto"])].sort_values("time")
fig = px.line(df, x="time", y="total", color='location')
fig.update_layout(height=800, width=1600, title_text="Accommodations of the top five most visited locations in Japan",  xaxis_title="Time", yaxis_title="Accommodations", font=dict(size=22))

fig.show()
#fig.write_html("newplot.html")
#fig.write_image("fig1.png")

In [48]:
table[(table["time"]  >= "2019-01-01") & (table["time"]  < "2020-01-01") ].groupby("location").sum(["total"]).nlargest(11, "total")

,total,foreigners
location,,
Japan,595921480,115656340
Tokyo,78981720,29350660
Osaka,47427520,17926180
Hokkaidou,36983420,8805160
Okinawa,32865650,7750770
Kyoto,30749560,12025050
Chiba,29229110,4798230
Kanagawa,23883900,3248690
Shizuoka,23429450,2493790


In [49]:
table["foreigner_ratio"] = 100 * table["foreigners"] / table["total"]

In [50]:
table[table["location"] == "Japan"].tail(30)

,time,location,total,foreigners,foreigner_ratio
140,2018-09-01,Japan,44439890,6804510,15.311717
141,2018-10-01,Japan,46582920,8244600,17.698762
142,2018-11-01,Japan,45282650,7609070,16.803500
143,2018-12-01,Japan,43967910,8068220,18.350247
144,2019-01-01,Japan,42684710,9208780,21.573955
145,2019-02-01,Japan,43539370,9276270,21.305476
146,2019-03-01,Japan,51147600,9515070,18.603160
147,2019-04-01,Japan,50718730,11284480,22.249138
148,2019-05-01,Japan,51402690,9727570,18.924243
149,2019-06-01,Japan,45810390,9586990,20.927545


In [51]:

df = table[table["location"] == "Japan"].sort_values("time")
fig = px.line(df, x="time", y="foreigner_ratio")
fig.update_layout(height=800, width=1600, title_text="Percentage of foreign visitors",  xaxis_title="Time", yaxis_title="Foreigner percentage (%)", font=dict(size=22))

fig.show()
#fig.write_html("newplot.html")
#fig.write_image("fig1.png")

In [55]:
prophet_foreign_ratio = table[table["location"] == "Japan"][["time", "foreigner_ratio"]]
prophet_foreign_ratio.columns = ["ds", "y"]
prophet_foreign_ratio["ds"] = pd.to_datetime(prophet_foreign_ratio["ds"])

prophet_foreign_ratio_2020 = prophet_foreign_ratio[(prophet_foreign_ratio["ds"] >= "2020-01-01")]
prophet_foreign_ratio_before_2020 = prophet_foreign_ratio[(prophet_foreign_ratio["ds"] < "2020-01-01")]
m_foreign_ratio = Prophet()
m_foreign_ratio.fit(prophet_foreign_ratio_before_2020)
future = pd.DataFrame(prophet_foreign_ratio["ds"], columns=["ds"])
prediction = m_foreign_ratio.predict(future)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [54]:
prophet_foreign_ratio

,ds,y
0,2007-01-01,6.556163
1,2007-02-01,7.494923
2,2007-03-01,6.607463
3,2007-04-01,8.875553
4,2007-05-01,7.157075
...,...,...
165,2020-10-01,0.820877
166,2020-11-01,1.258300
167,2020-12-01,1.809553
168,2021-01-01,2.646636


In [59]:
fig_1 = plot_prophet(prophet_foreign_ratio, prediction, "Percentage of foreign visitors", "Percentage (%)")
fig_1.show()